In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import random

In [212]:
url = 'https://www.amazon.com/s/ref=lp_16225006011_pg_2?rh=i:specialty-aps,n:!16225006011&page=2&ie=UTF8&qid=1534610352'
headers = {
    'cookie' : "session-id=137-4618154-3868331; session-id-time=2082787201l; csm-hit=tb:5PZ14JWAZ8ATN849YF0A+s-PVMZ7H1WFHZQPN19V4V6|1534612300081&adb:adblk_no; x-wl-uid=1/GkU3wlVStp842EK4jPL8yfJSL5rSD2bkhf7mzTBWBdsn41ShBApu+OkvqzqAJXZn9INUEyBq60=; ubid-main=135-1115950-8593103; session-token=qRmaSUtjqwt8VUj2vXaA0dLPauiCaOwbTd8BLwhpgpv4lMdON47PAsfjLFnD59MofWiFbdRfqHnSAkToYmQsGXqCe3xn2GLJxdOwG9IvHMz06GNsotNiTpteX2QPjQaWxhS0QBNin90pv3vhmrIBLGp43dFbgn3yaAhv6Ee/e+7YntcntDjl4xmYkdAFSca/KLYT9quhBCAFMotQEZPe18Kd8QI2UYOzMScYK07/w9IoTFsehHrcrOB1wQfb+L6dOJZO9ZdJhA0=",
    'User-Agent' : "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"
}

res = requests.get(url, headers = headers).text
res

In [217]:
# make soup
soup = BeautifulSoup(res,'lxml')
# product_title
# soup.select('#result_0')[0].select('.s-color-twister-title-link')[0]['title']
product_title = soup.select('#result_0')[0]#.select('.a-size-medium.s-inline.s-access-title.a-text-normal')[0].text.strip()
# print(product_title)

IndexError: list index out of range

In [126]:
# brand
brand = soup.select('#result_0')[0].select('.a-row.a-spacing-none')[1].select('.a-size-small.a-color-secondary')[1].text.strip()
print(brand)

COSRX


In [122]:
# original_price
p_int = soup.select('#result_0')[0].select('.sx-price.sx-price-large')[0].select('.sx-price-whole')[0].text
p_float = soup.select('#result_0')[0].select('.sx-price.sx-price-large')[0].select('.sx-price-fractional')[0].text
original_price = float('%s.%s'%(p_int,p_float))
print(original_price)

4.97


In [129]:
# discount_price
if soup.select('#result_0')[0].select('.a-text-strike'):
    discount_price = float(soup.select('#result_0')[0].select('.a-text-strike')[0].text[1:])
    print(discount_price)
else:
    print('no discount')

no discount


In [166]:
# review_num
# soup.select('#result_0')[0].select('.a-size-small.a-link-normal.a-text-normal')[0].text

# rating
t = soup.select('#result_0')[0].select('.a-icon.a-icon-star')[0].select('.a-icon-alt')[0].text
rating = float(re.match('(.+) out of 5 stars',t).group(1))
print('rating:',rating)

rating: 3.9


In [113]:
# best_seller
best_seller = False

if soup.select('#result_0')[0].select('.a-badge-text'):
    best_seller = True
    # best_in
    t = soup.select('#result_0')[0].select('.a-badge-supplementary-text.a-text-ellipsis')[0].text
    best_in = re.match('in (.+)',t).group(1).strip()
else:
    print('not best-seller')

print(best_seller, best_in)

True Beauty & Personal Care


In [141]:
# results_per_page
results = len(soup.select('.s-result-item.celwidget  '))
print(results)

16


In [167]:
# 一路翻到掛版本
url = 'https://www.amazon.com/s/ref=lp_16225006011_pg_2?rh=i:specialty-aps,n:!16225006011&page=%s&ie=UTF8&qid=1534610352'
headers = {
    'cookie' : "session-id=137-4618154-3868331; session-id-time=2082787201l; csm-hit=tb:5PZ14JWAZ8ATN849YF0A+s-PVMZ7H1WFHZQPN19V4V6|1534612300081&adb:adblk_no; x-wl-uid=1/GkU3wlVStp842EK4jPL8yfJSL5rSD2bkhf7mzTBWBdsn41ShBApu+OkvqzqAJXZn9INUEyBq60=; ubid-main=135-1115950-8593103; session-token=qRmaSUtjqwt8VUj2vXaA0dLPauiCaOwbTd8BLwhpgpv4lMdON47PAsfjLFnD59MofWiFbdRfqHnSAkToYmQsGXqCe3xn2GLJxdOwG9IvHMz06GNsotNiTpteX2QPjQaWxhS0QBNin90pv3vhmrIBLGp43dFbgn3yaAhv6Ee/e+7YntcntDjl4xmYkdAFSca/KLYT9quhBCAFMotQEZPe18Kd8QI2UYOzMScYK07/w9IoTFsehHrcrOB1wQfb+L6dOJZO9ZdJhA0=",
    'User-Agent' : "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"
}
# start page
page = 1
results = 0
products = []
search_url = []
while True:
    url = url%page
    res = requests.get(url, headers = headers).text
    if BeautifulSoup(res,'lxml'):
        soup = BeautifulSoup(res,'lxml')
        r = len(soup.select('.s-result-item.celwidget  '))
        resuts += r
    else:
        break
    # whole page
    for i in range(results-r,results):
        result = '#result_%s'%i
        # product_title
        product_title = soup.select(result)[0].select('.a-size-medium.s-inline.s-access-title.a-text-normal')[0].text.strip()
        print('product_title:',product_title)
        # brand
        brand = soup.select(result)[0].select('.a-row.a-spacing-none')[1].select('.a-size-small.a-color-secondary')[1].text.strip()
        print('brand:',brand)
        # original_price
        p_int = soup.select(result)[0].select('.sx-price.sx-price-large')[0].select('.sx-price-whole')[0].text
        p_float = soup.select(result)[0].select('.sx-price.sx-price-large')[0].select('.sx-price-fractional')[0].text
        original_price = float('%s.%s'%(p_int,p_float))
        print('original_price:',original_price)
        # discount_price
        discount_price = None
        if soup.select(result)[0].select('.a-text-strike'):
            discount_price = float(soup.select(result)[0].select('.a-text-strike')[0].text[1:])
            print('discount_price:',discount_price)
        else:
            print('no discount')
        # rating
        t = soup.select(result)[0].select('.a-icon.a-icon-star')[0].select('.a-icon-alt')[0].text
        rating = float(re.match('(.+) out of 5 stars',t).group(1))
        print('rating:',rating)
        # best_seller
        best_seller = False
        best_in = None
        if soup.select(result)[0].select('.a-badge-text'):
            best_seller = True
            print('is best-seller')
            # best_in
            t = soup.select(result)[0].select('.a-badge-supplementary-text.a-text-ellipsis')[0].text
            best_in = re.match('in (.+)',t).group(1).strip()
            print('best in',best_in)
        else:
            print('not best-seller')
        pd = {
            'product_title' : product_title,
            'brand' : brand,
            'original_price' : original_price,
            'discount_price' : discount_price,
            'rating' : rating,
            'best_seller' : best_seller,
            'best in' : best_in
        }
        # search_url
        search_url.append(soup.select('.a-link-normal.s-access-detail-page.s-color-twister-title-link.a-text-normal')[0].get('href'))
        print('============================================================')
        products.append(pd)
        # turning page
        page += 1
        # sleep 5
        s = random.randint(1,10)
        time.sleep(s)

product_title: COSRX Acne Pimple Master Patch, 24 Patches
brand: COSRX
original_price: 4.97
no discount
rating: 3.9
is best-seller
best in Beauty & Personal Care
product_title: Gillette Fusion Manual Men’s Razor Blade Refills, 12 Count, Mens Razors/Blades
brand: Gillette
original_price: 23.61
discount_price: 47.99
rating: 4.6
is best-seller
best in Men's Replacement Razor Blade Cartridges & Refills
product_title: Q-tips Cotton, Swabs, 500 ct, 4 pack
brand: Q-Tips
original_price: 11.36
discount_price: 32.99
rating: 4.5
is best-seller
best in Cotton Swabs
product_title: Philips Norelco Multi Groomer MG3750/50-13 piece, beard, face, nose, and ear hair trimmer and clipper, FFP
brand: Philips Norelco
original_price: 19.95
discount_price: 20.99
rating: 4.1
is best-seller
best in Personal Groomers
product_title: Crest 3D White Whitening Toothpaste, Radiant Mint, 3.5 oz, Pack of 2
brand: Crest
original_price: 5.71
discount_price: 7.49
rating: 4.4
is best-seller
best in Toothpaste
product_title

In [14]:
# 翻幾頁版本
url = 'https://www.amazon.com/s/ref=lp_16225006011_pg_2?rh=i:specialty-aps,n:!16225006011&page=%s&ie=UTF8&qid=1534610352'
headers = {
    'cookie' : "session-id=137-4618154-3868331; session-id-time=2082787201l; csm-hit=tb:5PZ14JWAZ8ATN849YF0A+s-PVMZ7H1WFHZQPN19V4V6|1534612300081&adb:adblk_no; x-wl-uid=1/GkU3wlVStp842EK4jPL8yfJSL5rSD2bkhf7mzTBWBdsn41ShBApu+OkvqzqAJXZn9INUEyBq60=; ubid-main=135-1115950-8593103; session-token=qRmaSUtjqwt8VUj2vXaA0dLPauiCaOwbTd8BLwhpgpv4lMdON47PAsfjLFnD59MofWiFbdRfqHnSAkToYmQsGXqCe3xn2GLJxdOwG9IvHMz06GNsotNiTpteX2QPjQaWxhS0QBNin90pv3vhmrIBLGp43dFbgn3yaAhv6Ee/e+7YntcntDjl4xmYkdAFSca/KLYT9quhBCAFMotQEZPe18Kd8QI2UYOzMScYK07/w9IoTFsehHrcrOB1wQfb+L6dOJZO9ZdJhA0=",
    'User-Agent' : "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"
}
# start page
search_url = []
products = []
results = 0
for p in range(1,3):
    u = url%p
    res = requests.get(u, headers = headers).text
    if BeautifulSoup(res,'lxml'):
        soup = BeautifulSoup(res,'lxml')
        r = len(soup.select('.s-result-item.celwidget  '))
        results += r
    else:
        break
    # whole page
    for i in range(results-r,results):
        result = '#result_%s'%i
        # product_title
        product_title = soup.select(result)[0].select('.a-size-medium.s-inline.s-access-title.a-text-normal')[0].text.strip()
        print('product_title:',product_title)
        # brand
        brand = soup.select(result)[0].select('.a-row.a-spacing-none')[1].select('.a-size-small.a-color-secondary')[1].text.strip()
        print('brand:',brand)
        # original_price
        if soup.select(result)[0].select('.sx-price.sx-price-large'):
            p_int = soup.select(result)[0].select('.sx-price.sx-price-large')[0].select('.sx-price-whole')[0].text
            p_float = soup.select(result)[0].select('.sx-price.sx-price-large')[0].select('.sx-price-fractional')[0].text
            original_price = float('%s.%s'%(p_int,p_float))
            print('original_price:',original_price)
        else:
            print('no price tag, pass this data')
            print('============================================================')
            continue
        # discount_price
        discount_price = None
        if soup.select(result)[0].select('.a-text-strike'):
            discount_price = float(soup.select(result)[0].select('.a-text-strike')[0].text[1:])
            print('discount_price:',discount_price)
        else:
            print('no discount')
        # rating
        t = soup.select(result)[0].select('.a-icon.a-icon-star')[0].select('.a-icon-alt')[0].text
        rating = float(re.match('(.+) out of 5 stars',t).group(1))
        print('rating:',rating)
        # best_seller
        best_seller = False
        best_in = None
        if soup.select(result)[0].select('.a-badge-text'):
            best_seller = True
            print('is best-seller')
            # best_in
            t = soup.select(result)[0].select('.a-badge-supplementary-text.a-text-ellipsis')[0].text
            best_in = re.match('in (.+)',t).group(1).strip()
            print('best in',best_in)
        else:
            print('not best-seller')
        pd = {
            'product_title' : product_title,
            'brand' : brand,
            'original_price' : original_price,
            'discount_price' : discount_price,
            'rating' : rating,
            'best_seller' : best_seller,
            'best in' : best_in
        }
        # search_url
        search_url.append(soup.select('.a-link-normal.s-access-detail-page.s-color-twister-title-link.a-text-normal')[0].get('href'))
        print('============================================================')
        products.append(pd)
        # sleep 5
      #  s = random.randint(1,10)
      #  time.sleep(s)

product_title: COSRX Acne Pimple Master Patch, 24 Patches
brand: COSRX
original_price: 4.97
no discount
rating: 3.9
is best-seller
best in Beauty & Personal Care
product_title: Gillette Fusion Manual Men’s Razor Blade Refills, 12 Count, Mens Razors/Blades
brand: Gillette
original_price: 23.61
discount_price: 47.99
rating: 4.5
is best-seller
best in Men's Replacement Razor Blade Cartridges & Refills
product_title: Q-tips Cotton, Swabs, 500 ct, 4 pack
brand: Q-Tips
original_price: 11.36
discount_price: 32.99
rating: 4.5
is best-seller
best in Cotton Swabs
product_title: Crest 3D White Whitening Toothpaste, Radiant Mint, 3.5 oz, Pack of 2
brand: Crest
original_price: 5.71
discount_price: 7.49
rating: 4.4
is best-seller
best in Toothpaste
product_title: Aztec Secret Indian Healing Clay Deep Pore Cleansing, 1 Pound
brand: Aztec Secret
original_price: 9.34
no discount
rating: 4.6
is best-seller
best in Body Scrubs & Treatments
product_title: Cali White ACTIVATED CHARCOAL & ORGANIC COCONUT OI

In [11]:
products

[{'product_title': 'COSRX Acne Pimple Master Patch, 24 Patches',
  'brand': 'COSRX',
  'original_price': 4.97,
  'discount_price': None,
  'rating': 3.9,
  'best_seller': True,
  'best in': 'Beauty & Personal Care'},
 {'product_title': 'Gillette Fusion Manual Men’s Razor Blade Refills, 12 Count, Mens Razors/Blades',
  'brand': 'Gillette',
  'original_price': 23.61,
  'discount_price': 47.99,
  'rating': 4.5,
  'best_seller': True,
  'best in': "Men's Replacement Razor Blade Cartridges & Refills"},
 {'product_title': 'Q-tips Cotton, Swabs, 500 ct, 4 pack',
  'brand': 'Q-Tips',
  'original_price': 11.36,
  'discount_price': 32.99,
  'rating': 4.5,
  'best_seller': True,
  'best in': 'Cotton Swabs'},
 {'product_title': 'Crest 3D White Whitening Toothpaste, Radiant Mint, 3.5 oz, Pack of 2',
  'brand': 'Crest',
  'original_price': 5.71,
  'discount_price': 7.49,
  'rating': 4.4,
  'best_seller': True,
  'best in': 'Toothpaste'},
 {'product_title': 'Aztec Secret Indian Healing Clay Deep Pore

In [16]:
len(search_url)

31